In [265]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests
import time
import hashlib
import hmac
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns


os.environ.pop("API_KEY")
os.environ.pop("SECRET_KEY")

load_dotenv()

API_KEY = os.environ["API_KEY"]
API_SECRET = os.environ["SECRET_KEY"]

# Helper Functions

In [266]:
def boilerplate(params, endpoint):
    BASE_URL = 'https://api.binance.com'

    # Create the signature using HMAC-SHA256
    query_string = '&'.join([f"{key}={value}" for key, value in params.items()])
    signature = hmac.new(
        API_SECRET.encode('utf-8'),
        query_string.encode('utf-8'),
        hashlib.sha256
    ).hexdigest()

    # Add the signature to the parameters
    params['signature'] = signature

    # Define the headers with API key
    headers = {
        'X-MBX-APIKEY': API_KEY
    }

    # Send the GET request
    response = requests.get(BASE_URL + endpoint, headers=headers, params=params)

    # Check response status and print the result
    if response.status_code == 200:
        # print(response.json())
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return Exception(f"Error: {response.status_code}, {response.text}")
    
def boilerplate1(params, endpoint):
    BASE_URL = 'https://api.binance.com'

    # Create the signature using HMAC-SHA256
    query_string = '&'.join([f"{key}={value}" for key, value in params.items()])
    # signature = hmac.new(
    #     API_SECRET.encode('utf-8'),
    #     query_string.encode('utf-8'),
    #     hashlib.sha256
    # ).hexdigest()

    # # Add the signature to the parameters
    # params['signature'] = signature

    # Define the headers with API key
    headers = {
        'X-MBX-APIKEY': API_KEY
    }

    # Send the GET request
    response = requests.get(BASE_URL + endpoint, headers=headers, params=params)

    # Check response status and print the result
    if response.status_code == 200:
        # print(response.json())
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return Exception(f"Error: {response.status_code}, {response.text}")

# Dual Investment Strategy
- **Put** Indicates that you are trying to buy the crypto product/ coin/ asset with stable coin. It means you are trying to buy Low
- **Call** Indicates that you are trying to Sell the crypto product/ coin/ asset for stable coin. It means you are trying to Sell High


In [267]:
def get_DCI_products(optionType, exercisedCoin, investCoin, pageSize, pageIndex):

    # Define the endpoint and base URL
    endpoint = '/sapi/v1/dci/product/list'

    # Define request parameters
    params = {
        'optionType': optionType,  #'CALL' or 'PUT'
        'exercisedCoin': exercisedCoin,  # Target exercised asset
        'investCoin': investCoin,  # Asset used for subscribing
        'pageSize': pageSize,  # Optional
        'pageIndex': pageIndex,  # Optional
        'recvWindow': 60000,  # Optional
        'timestamp': int(time.time() * 1000)  # Current timestamp in milliseconds
    }

    return boilerplate(params, endpoint)


In [268]:
def get_DCI_account():

    # Define the endpoint and base URL
    endpoint = '/sapi/v1/dci/product/accounts'

    # Define request parameters
    params = {
        'recvWindow': 60000,  # Optional
        'timestamp': int(time.time() * 1000)  # Current timestamp in milliseconds
    }

    return boilerplate(params, endpoint)



In [269]:
def get_DCI_positions():

    # Define the endpoint and base URL
    endpoint = '/sapi/v1/dci/product/positions'

    # Define request parameters
    params = {
        'recvWindow': 60000,  # Optional
        'timestamp': int(time.time() * 1000)  # Current timestamp in milliseconds
    }
    return boilerplate(params, endpoint)


In [270]:
def get_price_historical(symbol, interval):
    
    # Define the endpoint and base URL
    endpoint = '/api/v3/klines'

    # Define request parameters
    params = {
        'symbol': symbol,   
        'interval': interval  
    }
    return boilerplate1(params, endpoint)

# Runner


In [317]:
target = "ETH"
currency = "USDC"
target_currency = f"{target}{currency}"
print(target_currency)

ETHUSDC


In [318]:
data = get_price_historical(target_currency, "1d")
columns = [
    "Kline Open Time", "Open Price", "High Price", "Low Price", 
    "Close Price", "Volume", "Kline Close Time", "Quote Asset Volume", 
    "Number of Trades", "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume"
]

cleaned_data = [row[:-1] for row in data]  # Remove the last element from each row
df_history = pd.DataFrame(cleaned_data, columns=columns)

df_history['Kline Open Time'] = pd.to_datetime(df_history['Kline Open Time'],unit='ms')
df_history['Kline Close Time'] = pd.to_datetime(df_history['Kline Close Time'],unit='ms')

df_history["Open Price"] = pd.to_numeric(df_history["Open Price"])
df_history["High Price"] = pd.to_numeric(df_history["High Price"])
df_history["Low Price"] = pd.to_numeric(df_history["Low Price"])
df_history["Close Price"] = pd.to_numeric(df_history["Close Price"])
df_history["Volume"] = pd.to_numeric(df_history["Volume"])
df_history["Quote Asset Volume"] = pd.to_numeric(df_history["Quote Asset Volume"])
df_history["Taker Buy Base Asset Volume"] = pd.to_numeric(df_history["Taker Buy Base Asset Volume"])
df_history["Taker Buy Quote Asset Volume"] = pd.to_numeric(df_history["Taker Buy Quote Asset Volume"])



df_copy = df_history

In [319]:
curr_price = df_history.tail(1)["Close Price"].iloc[0]
curr_price

np.float64(2445.8)

In [320]:
df_price_movements_column = []
for n in range(1, 31):
    df_price_movements_column.append(f'Price Movement {n} days')
    df_copy[f'Price Movement {n} days'] = (df_copy['Close Price'].shift(n) - df_copy['Close Price']) / df_copy['Close Price']


df_price_movements = df_copy.dropna()
df_price_movements=df_price_movements[df_price_movements_column]
df_price_movements


,Price Movement 1 days,Price Movement 2 days,Price Movement 3 days,Price Movement 4 days,Price Movement 5 days,Price Movement 6 days,Price Movement 7 days,Price Movement 8 days,Price Movement 9 days,Price Movement 10 days,...,Price Movement 21 days,Price Movement 22 days,Price Movement 23 days,Price Movement 24 days,Price Movement 25 days,Price Movement 26 days,Price Movement 27 days,Price Movement 28 days,Price Movement 29 days,Price Movement 30 days
30,0.004495,-0.037822,-0.050230,-0.018186,-0.033826,-0.013136,-0.025331,-0.016108,-0.026957,-0.018415,...,-0.089496,-0.043927,-0.040945,-0.047876,-0.020872,-0.059079,-0.017521,-0.016674,-0.008641,-0.032402
31,-0.006361,-0.001895,-0.043942,-0.056271,-0.024431,-0.039972,-0.019413,-0.031530,-0.022366,-0.033146,...,-0.094901,-0.095288,-0.050009,-0.047045,-0.053932,-0.027100,-0.065064,-0.023770,-0.022929,-0.014947
32,-0.009573,-0.015872,-0.011449,-0.053094,-0.065305,-0.033770,-0.049162,-0.028799,-0.040801,-0.031724,...,-0.108806,-0.103565,-0.103948,-0.059102,-0.056167,-0.062989,-0.036414,-0.074013,-0.033115,-0.032282
33,0.009952,0.000284,-0.006079,-0.001611,-0.043670,-0.056003,-0.024154,-0.039699,-0.019134,-0.031255,...,-0.101770,-0.099937,-0.094643,-0.095031,-0.049739,-0.046774,-0.053664,-0.026824,-0.064798,-0.023493
34,0.013563,0.023649,0.013850,0.007402,0.011929,-0.030700,-0.043200,-0.010919,-0.026675,-0.005831,...,-0.135903,-0.089588,-0.087730,-0.082365,-0.082757,-0.036851,-0.033846,-0.040829,-0.013625,-0.052114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-0.006903,0.000143,-0.010395,-0.010313,-0.036854,-0.030984,0.003423,0.066423,0.089562,0.096879,...,-0.040002,-0.059033,-0.050511,-0.008649,0.000389,-0.031828,-0.040448,-0.020921,-0.032566,-0.058312
496,0.029875,0.022766,0.030023,0.019169,0.019254,-0.008080,-0.002035,0.033400,0.098283,0.122113,...,0.002394,-0.011322,-0.030922,-0.022145,0.020968,0.030276,-0.002904,-0.011782,0.008329,-0.003664
497,-0.006580,0.023099,0.016036,0.023245,0.012463,0.012547,-0.014606,-0.008601,0.026600,0.091056,...,0.034228,-0.004202,-0.017827,-0.037298,-0.028580,0.014250,0.023497,-0.009465,-0.018284,0.001694
498,-0.022268,-0.028702,0.000316,-0.006589,0.000459,-0.010082,-0.010000,-0.036550,-0.030678,0.003739,...,0.050281,0.011198,-0.026377,-0.039699,-0.058736,-0.050212,-0.008336,0.000705,-0.031523,-0.040145


In [321]:
summary_stats = df_price_movements.describe()
skewness = df_price_movements.skew()
kurtosis = df_price_movements.kurtosis()

In [322]:
# 1. Visualizing Price Movements Using Line Plot
fig_line = go.Figure()

# Create a line for each column
for col in df_price_movements.columns:
    fig_line.add_trace(go.Scatter(x=df_price_movements.index, y=df_price_movements[col], mode='lines', name=col))

# Update layout
fig_line.update_layout(
    title=f"Daily Price Movement Percentage {target}",
    xaxis_title="Time",
    yaxis_title="Percentage Movement",
    template='plotly'
)

# Show the line plot
fig_line.show()

# 2. Visualizing the Distribution with Box Plot
fig_box = px.box(df_price_movements, title=f"Distribution of Price Movements {target}")

# Show the box plot
fig_box.show()

## Dual Investment tracking

In [323]:
TargetItem = target
AssetItem = currency
Direction = "PUT"

first = get_DCI_products(Direction, TargetItem, AssetItem, pageIndex=1, pageSize=1)
full_list = first["list"]
total = first["total"]
print(total)

for i in range(total//100+1):
    try:
        curr = get_DCI_products(Direction, TargetItem, AssetItem, pageIndex=i+1, pageSize=100)
        full_list.extend(curr["list"])
    except Exception as e:
        print(f"ERROR found: {e}")

print("---------------------------")
print(len(full_list))

330
---------------------------
331


In [324]:
df = pd.DataFrame(full_list)
df["curr_price"]  = curr_price
df["strikePrice"] = pd.to_numeric(df["strikePrice"])

df["Percent_to_strikeprice"] = 100*(df["strikePrice"] - df["curr_price"])/df["curr_price"]
# df["percentage_over_"]

In [325]:
df = df.sort_values(by=['apr', 'duration'], ascending=[False, True])
df["strikePrice"] = df.strikePrice.astype(float)
df["apr"] = df.apr.astype(float)
df["1000return"] = df["apr"]*df["duration"]*1000/365

df

,id,investCoin,exercisedCoin,strikePrice,duration,settleDate,purchaseDecimal,purchaseEndTime,canPurchase,apr,orderId,minAmount,maxAmount,createTimestamp,optionType,isAutoCompoundEnable,autoCompoundPlanList,curr_price,Percent_to_strikeprice,1000return
0,1366989,USDC,ETH,2425.0,3,1728979200000,8,1728892800000,True,0.8794,19454556973,0.1,5001671,1727425645000,PUT,True,"[STANDARD, ADVANCE]",2445.8,-0.850437,7.227945
1,1366989,USDC,ETH,2425.0,3,1728979200000,8,1728892800000,True,0.8794,19454556973,0.1,5001671,1727425645000,PUT,True,"[STANDARD, ADVANCE]",2445.8,-0.850437,7.227945
2,1306692,USDC,ETH,2425.0,6,1729238400000,8,1729152000000,True,0.8538,19454524604,0.1,1697500,1725156144000,PUT,True,"[STANDARD, ADVANCE]",2445.8,-0.850437,14.035068
3,1213346,USDC,ETH,2425.0,13,1729843200000,8,1729756800000,True,0.6726,19448529739,0.1,4800000,1721894844000,PUT,True,"[STANDARD, ADVANCE]",2445.8,-0.850437,23.955616
4,1306430,USDC,ETH,2400.0,6,1729238400000,8,1729152000000,True,0.6346,19454524247,0.1,1680000,1725156144000,PUT,True,"[STANDARD, ADVANCE]",2445.8,-1.872598,10.431781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,1341243,USDC,ETH,1725.0,111,1738310400000,8,1738224000000,True,0.0373,19441801027,0.1,4800000,1726725627000,PUT,True,"[STANDARD, ADVANCE]",2445.8,-29.470930,11.343288
330,1389742,USDC,ETH,1900.0,20,1730448000000,8,1730361600000,True,0.0371,19452561559,0.1,1520000,1727765531000,PUT,True,"[STANDARD, ADVANCE]",2445.8,-22.315807,2.032877
321,1240237,USDC,ETH,1750.0,111,1738310400000,8,1738224000000,True,0.0369,19441799816,0.1,4800000,1722729623000,PUT,True,"[STANDARD, ADVANCE]",2445.8,-28.448769,11.221644
322,1246812,USDC,ETH,1950.0,13,1729843200000,8,1729756800000,True,0.0366,19454553168,0.1,5001671,1722784321000,PUT,True,"[STANDARD, ADVANCE]",2445.8,-20.271486,1.303562


In [326]:
df.dtypes

id                         object
investCoin                 object
exercisedCoin              object
strikePrice               float64
duration                    int64
settleDate                  int64
purchaseDecimal             int64
purchaseEndTime             int64
canPurchase                  bool
apr                       float64
orderId                     int64
minAmount                  object
maxAmount                  object
createTimestamp             int64
optionType                 object
isAutoCompoundEnable         bool
autoCompoundPlanList       object
curr_price                float64
Percent_to_strikeprice    float64
1000return                float64
dtype: object

In [327]:
fig = px.scatter(df, x="Percent_to_strikeprice", y="apr", hover_data=["id","orderId", "duration", "1000return"], title=f"{TargetItem} Buy low target")
fig.update_layout(
    yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 0.1
    )
)
fig.show()


In [328]:
fig = px.scatter(df, x="strikePrice", y="apr", hover_data=["id","orderId", "duration", "1000return"], title=f"{TargetItem} Buy low target")
fig.update_layout(
    yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 0.1
    )
)
fig.show()


In [329]:
fig = px.scatter(df, x="duration", y="apr", hover_data=["strikePrice", "id", "1000return"])
fig.show()

In [330]:
fig = px.scatter(df, x="duration", y="1000return", color="strikePrice", hover_data=["id", "apr", "strikePrice"])
fig.show()

In [331]:
fig = px.scatter(df, x="strikePrice", y="1000return", hover_data=["id", "apr", "strikePrice", "duration"])
fig.show()

In [358]:
df_copy = df[df["strikePrice"]<=2320]

df_copy = df_copy[df_copy["strikePrice"]>=2250]
df_copy = df_copy[df_copy["duration"]<=25]
# df_copy = df[df["duration"]>=0]
fig = px.scatter(df_copy, x="strikePrice", y="duration",color="apr", size="apr", hover_data=["id", "apr", "strikePrice", "duration", "1000return", "Percent_to_strikeprice"])



fig.show()



In [367]:
df_copy = df[df["strikePrice"]>=2200]
df_copy = df_copy[df_copy["duration"]<=25]
df_copy = df_copy[df_copy["apr"]>=0.2]


fig = px.scatter(df_copy, x="duration", y="apr", color='strikePrice')
fig.show()

In [359]:
10.1435*8.4145

85.35248075

In [ ]:
1213342

In [333]:
get_DCI_account()

{'totalAmountInBTC': '0.02474803', 'totalAmountInUSDT': '1553.43331006'}

In [334]:
get_DCI_positions()

{'total': 5,
 'list': [{'id': '12042354',
   'investCoin': 'USDC',
   'exercisedCoin': 'ETH',
   'subscriptionAmount': '8379.3',
   'strikePrice': '2275',
   'duration': 5,
   'settleDate': 1728633600000,
   'purchaseStatus': 'SETTLED',
   'apr': '0.3088',
   'orderId': 19205411832,
   'purchaseEndTime': 1728547200000,
   'optionType': 'PUT',
   'settlePrice': '2407.00516667',
   'isExercised': False,
   'settleAsset': 'USDC',
   'settleAmount': '8414.49306'},
  {'id': '12001298',
   'investCoin': 'BTC',
   'exercisedCoin': 'FDUSD',
   'subscriptionAmount': '0.07445',
   'strikePrice': '68750',
   'duration': 3,
   'settleDate': 1728028800000,
   'purchaseStatus': 'SETTLED',
   'apr': '0.064',
   'orderId': 18975268294,
   'purchaseEndTime': 1727942400000,
   'optionType': 'CALL',
   'autoCompoundPlan': 'NONE',
   'settlePrice': '61452.1535',
   'isExercised': False,
   'settleAsset': 'BTC',
   'settleAmount': '0.07448722'},
  {'id': '11921394',
   'investCoin': 'FDUSD',
   'exercisedC